In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mt
import matplotlib.pyplot as pt
from keras.layers import BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics         import accuracy_score
from numpy import expand_dims
from tensorflow import keras
import tensorflow as tf
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randn
from numpy.random import randint

import tensorflow as tf
from keras.optimizers import Adam
import keras
from keras import layers as lys
from keras import models as mls
from keras import initializers
from keras.utils import to_categorical as ct
import tensorflow as tf
from tensorflow.keras import layers, models, initializers, Input
from keras.layers import Dense, Embedding, Flatten, Concatenate
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.models import Model
import io
import gdown
import scipy.io

In [ ]:
import gdown

# File ID from Google Drive
file_id = "1Z_mEDB60-BnoKCEbiYqxKchk4NSgLcKW"

# Download the file
gdown.download(f"https://drive.google.com/uc?id={file_id}", "PV_Data.xlsx", quiet=False)

# Load the Excel file
df = pd.read_excel("PV_Data.xlsx")

Downloading...
From: https://drive.google.com/uc?id=1Z_mEDB60-BnoKCEbiYqxKchk4NSgLcKW
To: /content/PV_Data.xlsx
100%|██████████| 396k/396k [00:00<00:00, 85.7MB/s]


In [ ]:

np.random.seed(42)
data = df.to_numpy()
input = data[:, 1:8]
output = data[:, 0]
output = output - 1

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
minmax = MinMaxScaler()
X =  minmax.fit_transform(input)
pca = PCA(n_components=7)
pca.fit(X)
X= pca.transform(X)
Y = output

In [ ]:
np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=6)
print(X_test.shape)
print(y_test.shape)
print('**********')
print(X_train.shape)
print(y_train.shape)

(350, 7)
(350,)
**********
(3150, 7)
(3150,)


In [ ]:
import pickle

# Ensure X_train, X_test, y_train, y_test exist before saving
data = ((X_train, y_train), (X_test, y_test))

# Save to a file
with open('custom_fashion_mnist.pkl', 'wb') as f:
    pickle.dump(data, f)

print("Dataset saved successfully!")

Dataset saved successfully!


In [ ]:
def load_PVdata():
    with open('custom_fashion_mnist.pkl', 'rb') as f:
        data = pickle.load(f)
    return data

In [ ]:
def define_discriminator(in_shape=(7,), n_classes=10):
  # image input
  in_image = Input(shape=in_shape)
  # downsample
  fe = Dense(units=180, activation='sigmoid',kernel_initializer=tf.keras.initializers.GlorotNormal(),
    bias_initializer=initializers.Zeros())(in_image)
  fe = LeakyReLU(negative_slope=0.4)(fe)
  # downsample
  fe = Dense(units=140, activation='sigmoid',  kernel_initializer=tf.keras.initializers.GlorotNormal(),
    bias_initializer=initializers.Zeros())(fe)
  fe = LeakyReLU(negative_slope=0.4)(fe)
  # downsample
  fe = Dense(units=120, activation='sigmoid',  kernel_initializer=tf.keras.initializers.GlorotNormal(),
    bias_initializer=initializers.Zeros())(fe)
  fe = LeakyReLU(negative_slope=0.4)(fe)
  # dropout
  fe = Dropout(0.4)(fe)
  # unsupervised output
  out1 = Dense(1, activation = 'sigmoid')(fe)
  # supervised output
  out2 = Dense(n_classes, activation='softmax')(fe)
  # define and compile supervised discriminator model
  model = Model(in_image, [out1, out2])
  opt =  optimizer=Adam(learning_rate= keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.002,decay_steps=10000,decay_rate=0.9),beta_1=0.5)
  model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer = opt)
  return model

model = define_discriminator()
# summarize the model
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6             │ (None, 7)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_16 (Dense)          │ (None, 180)            │          1,440 │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_6 (LeakyReLU) │ (None, 180)            │              0 │ dense_16[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_17 (Dense)          │ (None, 140)            │         25,340 │ leaky_re_lu_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_7 (LeakyReLU) │ (None, 140)            │              0 │ dense_17[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_18 (Dense)          │ (None, 120)            │         16,920 │ leaky_re_lu_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_8 (LeakyReLU) │ (None, 120)            │              0 │ dense_18[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 120)            │              0 │ leaky_re_lu_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_19 (Dense)          │ (None, 1)              │            121 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_20 (Dense)          │ (None, 10)             │          1,210 │ dropout_2[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 45,031 (175.90 KB)

 Trainable params: 45,031 (175.90 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
def define_generator(latent_dim = 100, n_outputs=7, n_classes = 10):
    in_label = Input(shape=(1,))
    li = Embedding(n_classes, 10)(in_label)
    li = Flatten()(li)
    # Latent space input
    in_lat = Input(shape=(latent_dim,))

    # Merge latent vector and label embedding
    merged = Concatenate()([in_lat, li])

    fe = Dense(180, activation='tanh', kernel_initializer=tf.keras.initializers.GlorotNormal(),bias_initializer=initializers.Zeros())(merged)
    fe = Dense(100, activation='tanh', kernel_initializer=tf.keras.initializers.GlorotNormal(),bias_initializer=initializers.Zeros())(fe)
    out_layer = Dense(n_outputs, activation='relu')(fe)
    model = Model([in_lat, in_label], out_layer)
    return model

model = define_generator()
# summarize the model
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 1, 10)          │            100 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_8             │ (None, 100)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_2 (Flatten)       │ (None, 10)             │              0 │ embedding_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 110)            │              0 │ input_layer_8[0][0],   │
│ (Concatenate)             │                        │                │ flatten_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_21 (Dense)          │ (None, 180)            │         19,980 │ concatenate_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_22 (Dense)          │ (None, 100)            │         18,100 │ dense_21[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_23 (Dense)          │ (None, 7)              │            707 │ dense_22[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 38,887 (151.90 KB)

 Trainable params: 38,887 (151.90 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# example of fitting an auxiliary classifier gan (ac-gan) on fashion mnsit
from numpy import zeros
from numpy import ones
from numpy import expand_dims
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Activation
from keras.layers import Concatenate
from keras.initializers import RandomNormal
from matplotlib import pyplot


# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	#for layer in d_model.layers:
	#	if not isinstance(layer, BatchNormalization):
	#		layer.trainable = False
	# connect the outputs of the generator to the inputs of the discriminator
	gan_output = d_model(g_model.output)
	# define gan model as taking noise and label and outputting real/fake and label outputs
	model = Model(g_model.input, gan_output)
	# compile model
	opt = Adam(learning_rate=0.0002, beta_1=0.5)
	model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
	return model

# load images
def load_real_samples():
	# load dataset
	(trainX, trainy), (_, _) = load_PVdata()
	# expand to 3d, e.g. add channels
	#X = expand_dims(trainX, axis=-1)
	# convert from ints to floats
	#X = X.astype('float32')
	# scale from [0,255] to [-1,1]
	#X = (X - 127.5) / 127.5
	print(trainX.shape, trainy.shape)
	return [trainX, trainy]

# select real samples
def generate_real_samples(dataset, n_samples):
	# split into images and labels
	images, labels = dataset
	# choose random instances
	ix = randint(0, images.shape[0], n_samples)
	# select images and labels
	X, labels = images[ix], labels[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return [X, labels], y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=10):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	return [z_input, labels]

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	z_input, labels_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	images = generator.predict([z_input, labels_input])
	# create class labels
	y = zeros((n_samples, 1))
	return [images, labels_input], y

# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
	# prepare fake examples
	[X, _], _ = generate_fake_samples(g_model, latent_dim, n_samples)
	# scale from [-1,1] to [0,1]
	#X = (X + 1) / 2.0
	# plot images
	for i in range(100):
		# define subplot
		pyplot.subplot(10, 10, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		#pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
	# save plot to file
	filename1 = 'generated_plot_%04d.png' % (step+1)
	pyplot.savefig(filename1)
	pyplot.close()
	# save the generator model
	filename2 = 'model_%04d.h5' % (step+1)
	g_model.save(filename2)
	print('>Saved: %s and %s' % (filename1, filename2))

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=2200, n_batch=512):
	# calculate the number of batches per training epoch
	bat_per_epo = int(dataset[0].shape[0] / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# calculate the size of half a batch of samples
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_steps):
		# get randomly selected 'real' samples
		[X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
		# update discriminator model weights
		_,d_r1,d_r2 = d_model.train_on_batch(X_real, [y_real, labels_real])
		# generate 'fake' examples
		[X_fake, labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
		# update discriminator model weights
		_,d_f,d_f2 = d_model.train_on_batch(X_fake, [y_fake, labels_fake])
		# prepare points in latent space as input for the generator
		[z_input, z_labels] = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = ones((n_batch, 1))
		# update the generator via the discriminator's error
		_,g_1,g_2 = gan_model.train_on_batch([z_input, z_labels], [y_gan, z_labels])
		# summarize loss on this batch
		print('>%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]' % (i+1, d_r1,d_r2, d_f,d_f2, g_1,g_2))
		# evaluate the model performance every 'epoch'
		if (i+1) % (bat_per_epo * 10) == 0:
			summarize_performance(i, g_model, latent_dim)

# size of the latent space
latent_dim = 100
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = load_real_samples()
# train model
train(generator, discriminator, gan_model, dataset, latent_dim)

(3150, 7) (3150,)
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>1, dr[0.381,2.567], df[1.082,2.594], g[2.491,2.491]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>2, dr[0.823,2.554], df[0.969,2.529], g[2.472,2.472]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>3, dr[0.865,2.506], df[0.907,2.487], g[2.463,2.463]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>4, dr[0.858,2.472], df[0.865,2.472], g[2.458,2.458]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>5, dr[0.839,2.463], df[0.842,2.463], g[2.451,2.451]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>6, dr[0.829,2.464], df[0.827,2.461], g[2.444,2.444]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>7, dr[0.817,2.456], df[0.818,2.450], g[2.433,2.433]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>8, dr[0.809,2.447], df[0.812,2.448], g[2.433,2.433]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>9, dr[0.806,2.444], df[0.808,2.440], g[2.427,2.427]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>10, dr[0.802,2.438], df[0.805,2.437], g[2.422,2.422]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>11, dr[0.798,2.437], df[0.802,2.

>Saved: generated_plot_0060.png and model_0060.h5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>61, dr[0.755,2.356], df[0.755,2.356], g[2.355,2.355]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>62, dr[0.755,2.355], df[0.755,2.355], g[2.354,2.354]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>63, dr[0.754,2.355], df[0.754,2.354], g[2.354,2.354]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>64, dr[0.754,2.354], df[0.754,2.354], g[2.353,2.353]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>65, dr[0.754,2.353], df[0.754,2.353], g[2.352,2.352]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>66, dr[0.753,2.353], df[0.753,2.353], g[2.352,2.352]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>67, dr[0.753,2.352], df[0.753,2.352], g[2.351,2.351]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>68, dr[0.752,2.351], df[0.753,2.351], g[2.351,2.351]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>69, dr[0.752,2.351], df[0.752,2.351], g[2.350,2.350]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>70, dr[0.752,2.350], df[0.752,2.350], g[2.349,2.349]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5m

>Saved: generated_plot_0120.png and model_0120.h5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>121, dr[0.737,2.333], df[0.737,2.332], g[2.333,2.333]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>122, dr[0.737,2.332], df[0.737,2.332], g[2.332,2.332]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>123, dr[0.737,2.332], df[0.737,2.332], g[2.332,2.332]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>124, dr[0.736,2.332], df[0.736,2.332], g[2.332,2.332]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>125, dr[0.736,2.332], df[0.736,2.332], g[2.332,2.332]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>126, dr[0.736,2.332], df[0.736,2.331], g[2.332,2.332]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>127, dr[0.736,2.331], df[0.736,2.331], g[2.331,2.331]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>128, dr[0.735,2.331], df[0.735,2.331], g[2.331,2.331]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>129, dr[0.735,2.331], df[0.735,2.331], g[2.331,2.331]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>130, dr[0.735,2.331], df[0.735,2.331], g[2.331,2.331]
8/8 ━━━━━━━━━━━━━━━━

>Saved: generated_plot_0180.png and model_0180.h5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>181, dr[0.727,2.323], df[0.727,2.323], g[2.324,2.324]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>182, dr[0.727,2.323], df[0.727,2.323], g[2.323,2.323]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>183, dr[0.727,2.323], df[0.727,2.323], g[2.323,2.323]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>184, dr[0.726,2.323], df[0.726,2.323], g[2.323,2.323]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>185, dr[0.726,2.323], df[0.726,2.322], g[2.323,2.323]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>186, dr[0.726,2.322], df[0.726,2.322], g[2.323,2.323]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>187, dr[0.726,2.322], df[0.726,2.322], g[2.323,2.323]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>188, dr[0.726,2.322], df[0.726,2.322], g[2.323,2.323]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>189, dr[0.726,2.322], df[0.726,2.322], g[2.323,2.323]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>190, dr[0.726,2.322], df[0.726,2.322], g[2.323,2.323]
8/8 ━━━━━━━━━━━━━━━━

>Saved: generated_plot_0240.png and model_0240.h5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>241, dr[0.721,2.317], df[0.721,2.317], g[2.319,2.319]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>242, dr[0.721,2.317], df[0.721,2.317], g[2.319,2.319]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>243, dr[0.720,2.317], df[0.721,2.316], g[2.318,2.318]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>244, dr[0.720,2.316], df[0.720,2.316], g[2.318,2.318]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>245, dr[0.720,2.316], df[0.720,2.316], g[2.318,2.318]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>246, dr[0.720,2.316], df[0.720,2.316], g[2.318,2.318]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>247, dr[0.720,2.316], df[0.720,2.316], g[2.318,2.318]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>248, dr[0.720,2.315], df[0.720,2.315], g[2.318,2.318]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
>249, dr[0.720,2.315], df[0.720,2.315], g[2.318,2.318]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>250, dr[0.720,2.315], df[0.720,2.315], g[2.317,2.317]
8/8 ━━━━━━━━━━━━━━━━

...

>Saved: generated_plot_3900.png and model_3900.h5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>3901, dr[0.304,0.791], df[0.304,0.791], g[0.819,0.819]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>3902, dr[0.304,0.791], df[0.304,0.791], g[0.818,0.818]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>3903, dr[0.304,0.791], df[0.304,0.791], g[0.818,0.818]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>3904, dr[0.304,0.791], df[0.304,0.791], g[0.818,0.818]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>3905, dr[0.304,0.791], df[0.304,0.791], g[0.818,0.818]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>3906, dr[0.304,0.791], df[0.304,0.791], g[0.818,0.818]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
>3907, dr[0.304,0.791], df[0.304,0.791], g[0.818,0.818]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>3908, dr[0.304,0.790], df[0.304,0.790], g[0.817,0.817]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
>3909, dr[0.304,0.790], df[0.304,0.790], g[0.817,0.817]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
>3910, dr[0.304,0.790], df[0.304,0.790], g[0.817,0.817]
8/8 ━━━━━━

>Saved: generated_plot_3960.png and model_3960.h5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>3961, dr[0.301,0.784], df[0.301,0.784], g[0.810,0.810]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>3962, dr[0.301,0.784], df[0.301,0.784], g[0.810,0.810]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>3963, dr[0.301,0.784], df[0.301,0.783], g[0.809,0.809]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>3964, dr[0.301,0.783], df[0.301,0.783], g[0.809,0.809]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>3965, dr[0.301,0.783], df[0.301,0.783], g[0.809,0.809]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>3966, dr[0.301,0.783], df[0.301,0.783], g[0.809,0.809]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
>3967, dr[0.301,0.783], df[0.301,0.783], g[0.809,0.809]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
>3968, dr[0.301,0.783], df[0.301,0.783], g[0.809,0.809]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
>3969, dr[0.301,0.783], df[0.301,0.783], g[0.809,0.809]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
>3970, dr[0.301,0.783], df[0.301,0.783], g[0.808,0.808]
8/8 ━━━━━━

>Saved: generated_plot_4020.png and model_4020.h5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>4021, dr[0.298,0.777], df[0.298,0.777], g[0.801,0.801]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
>4022, dr[0.298,0.777], df[0.298,0.777], g[0.801,0.801]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>4023, dr[0.298,0.776], df[0.298,0.776], g[0.801,0.801]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>4024, dr[0.298,0.776], df[0.298,0.776], g[0.801,0.801]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>4025, dr[0.298,0.776], df[0.298,0.776], g[0.801,0.801]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>4026, dr[0.298,0.776], df[0.298,0.776], g[0.801,0.801]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
>4027, dr[0.298,0.776], df[0.298,0.776], g[0.801,0.801]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>4028, dr[0.298,0.776], df[0.298,0.776], g[0.801,0.801]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>4029, dr[0.298,0.776], df[0.298,0.776], g[0.800,0.800]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>4030, dr[0.298,0.776], df[0.298,0.776], g[0.800,0.800]
8/8 ━━━━━━

>Saved: generated_plot_4080.png and model_4080.h5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>4081, dr[0.296,0.771], df[0.296,0.771], g[0.795,0.795]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>4082, dr[0.296,0.771], df[0.296,0.771], g[0.794,0.794]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
>4083, dr[0.296,0.771], df[0.296,0.771], g[0.794,0.794]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>4084, dr[0.296,0.771], df[0.296,0.771], g[0.794,0.794]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
>4085, dr[0.296,0.771], df[0.296,0.771], g[0.794,0.794]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>4086, dr[0.296,0.771], df[0.296,0.771], g[0.794,0.794]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>4087, dr[0.296,0.771], df[0.296,0.771], g[0.794,0.794]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>4088, dr[0.296,0.771], df[0.296,0.771], g[0.794,0.794]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
>4089, dr[0.296,0.771], df[0.296,0.770], g[0.794,0.794]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
>4090, dr[0.296,0.770], df[0.296,0.770], g[0.794,0.794]
8/8 ━━━━━━

>Saved: generated_plot_4140.png and model_4140.h5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>4141, dr[0.294,0.766], df[0.294,0.766], g[0.788,0.788]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>4142, dr[0.294,0.766], df[0.294,0.765], g[0.788,0.788]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>4143, dr[0.294,0.765], df[0.294,0.765], g[0.788,0.788]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
>4144, dr[0.294,0.765], df[0.294,0.765], g[0.788,0.788]


In [ ]:
from matplotlib import pyplot as plt
actual =y_test
feak_real, y_pred_test= discriminator.predict(X_test)
y_pred_test = np.argmax(y_pred_test, axis = 1)
confusion_matrix = metrics.confusion_matrix(actual, y_pred_test)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [0,1,2,3,4,5,6,7,8,9])
cm_display.plot()
plt.show()
confusion_matrix_per_class = confusion_matrix.astype('float') / confusion_matrix.sum(axis=1, keepdims=True) * 100
cm_display_per_class = metrics.ConfusionMatrixDisplay(confusion_matrix=confusion_matrix_per_class,display_labels=[0,1,2,3,4,5,6,7,8,9])
cm_display_per_class.plot(cmap="Blues", values_format=".2f")
plt.title("Confusion Matrix (Percentage Per Class)")
plt.show()

acc_test=accuracy_score(y_test, y_pred_test)
print("Accuracy Test: ", acc_test*100)